# LCEL(LangChain Expression Language)

- LCEL : 프롬프트 → LLM —> chain —> 프롬프트로 연결됨
- LCEL의 목표는 LangChain 사용자가 복잡한 데이터 흐름과 AI 작업을 보다 직관적으로 구현할 수 있도록 하는 것

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

# 프롬프트 정의

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template('{city}에서 가장 유명한 랜드마크는 무엇인가요?')


In [3]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({'city':'파리'})

C:\Users\magpi\AppData\Local\Temp\ipykernel_11832\2676035287.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'city': '파리',
 'text': '파리에서 가장 유명한 랜드마크는 에펠탑(Eiffel Tower)입니다. 에펠탑은 프랑스의 수도 파리에서 가장 아이코닉한 건축물 중 하나로, 세계적인 관광 명소로 유명합니다. 처음에는 역핑 탑(Édouard is a boy tower)로 불렸으나 현재의 이름은 건축가인 게스트라비에 실은 에펠리의 이름을 따서 명명되었다.'}

- LCEL(LCEL(LangChain Expression Language)) 형태로 전환함

In [4]:
chain = prompt | llm
chain.invoke({'city':'파리'})

AIMessage(content='파리에서 가장 유명한 랜드마크는 에펠탑(Eiffel Tower)입니다. 에펠탑은 프랑스의 수도인 파리의 상징적인 건물로, 세계적으로 유명한 관광 명소 중 하나로 손꼽힙니다. 에펠탑은 1889년에 완공되어 역사적인 건축물로서도 손꼽히며, 매년 수백만 명의 관광객이 방문하는 인기 있는 명소입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 33, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BfjMgLGSS4lR7WFnXY6c4CUyDsUyq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8384c00f-4002-4ff6-bd7c-f766dea31595-0', usage_metadata={'input_tokens': 33, 'output_tokens': 165, 'total_tokens': 198, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# SequentialChain 객체 활용

- 첫번째 프롬프트의 결과 값이 두 번째 프롬프트의 입력으로 들어가야 하는 것이 기본인데, 두 번 호출하지 않고 하나의 chain으로 만들 수 있습니다.

In [ ]:
from langchain.chains import SequentialChain

# citydhk transport 는 결과
# landmark는 첫번째 prompt의 결과값으로 사용해야 하므로, 
# 가져올 때 오직 지역의 랜드마크 이름만 가져오도록 만들어야 함
prompt_1 = PromptTemplate.from_template('{city}에서 가장 유명한 랜드마크는 무엇인가요? 설명은 필요없고 딱 이름 하나만 알려주세요')
prompt_2 = PromptTemplate.from_template('{landmark}에 {transport}로 가려면 어떻게 가나요?')

# prompt_1은 city를 입력으로 받고, landmark를 출력, 그 output_key가 protmpt_2의 landmark에 대입됨
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='landmark', verbose=True) #verbose=True로 하면 결과값을 출력해줌
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [6]:
chain = SequentialChain(chains=[chain_1, chain_2], 
                         input_variables=['city', 'transport'])

# 딕셔너리로 넣기
response = chain.invoke({'city':'서울', 'transport':'지하철'})



> Entering new LLMChain chain...
Prompt after formatting:
서울에서 가장 유명한 랜드마크는 무엇인가요? 설명은 필요없고 딱 이름 하나만 알려주세요

> Finished chain.


In [8]:
response

{'city': '서울',
 'transport': '지하철',
 'text': '롯데타워에 지하철로 가려면 먼저 가고자 하는 출발지와 롯데타워 강남역까지의 거리와 최적 경로를 확인해야 합니다.\n\n예를 들어, 지하철을 이용해 롯데타워 강남역에 도착하려면 먼저 각 출발역에서 강남역까지 가는 지하철 노선을 확인하고 강남역까지의 이동 방법을 선택해야 합니다. 일반적으로 서울 지하철은 1~9호선 및 공항철도, 삼성역과 역삼역을 경유하는 2호선 등이 있습니다.\n\n출발지와 강남역 간의 거리와 이용할 수 있는 지하철 노선을 고려하여 최적의 경로를 선택해 롯데타워 강남역까지 편리하게 이동할 수 있습니다.또는 네이버지도나 카카오 지도를 통해 대중교통으로 롯데타워로 가는 법을 찾을 수도 있습니다.'}